In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from scipy.optimize import curve_fit
from PIL import Image
import random

In [2]:
def leaky_relu(inp, alpha=0.2):
    return tf.maximum(inp * alpha, inp)

In [346]:
no_of_samples=30000
sample_length=500

samples=[''.join(random.choices(['0', '1', '2', '3'], k=sample_length)) for n in range(0,no_of_samples)]

In [347]:
needle_size=8

needle=''.join(random.choices(['0','1','2','3'], k=needle_size))

insert_count=15000
idx=np.random.randint(0,sample_length-needle_size-1,size=insert_count)
for i in range(0,insert_count):
    samples[i]=samples[i][0:idx[i]]+needle+samples[i][idx[i]+needle_size:]

truth=np.array([needle in samples[n] for n in range(0,len(samples))]).astype(int)
truth=np.reshape(truth, [truth.shape[0],1])
np.sum(truth)

15111

In [348]:
data=np.stack([[ np.eye(4)[int(samples[n][m])] for m in range(0,sample_length)] for n in range(0,len(samples))])
data=np.reshape(data, [data.shape[0], data.shape[1], 1, data.shape[2]])

In [349]:
samples_test=[''.join(random.choices(['0', '1', '2', '3'], k=sample_length)) for n in range(0,1000)]

insert_count=500
idx=np.random.randint(0,sample_length-needle_size-1,size=insert_count)
for i in range(0,insert_count):
    samples_test[i]=samples_test[i][0:idx[i]]+needle+samples_test[i][idx[i]+needle_size:]

truth_test=np.array([needle in samples_test[n] for n in range(0,len(samples_test))]).astype(int)
data_test=np.stack([[ np.eye(4)[int(samples_test[n][m])] for m in range(0,sample_length)] for n in range(0,len(samples_test))])

truth_test=np.reshape(truth_test, [truth_test.shape[0],1])
data_test=np.reshape(data_test, [data_test.shape[0], data_test.shape[1], 1, data_test.shape[2]])

In [350]:
tf.reset_default_graph()
#Input and output
x=tf.placeholder(dtype=tf.float32, shape=[None, sample_length,1,4])
y=tf.placeholder(dtype=tf.float32, shape=[None,1])

lr=tf.placeholder(dtype=tf.float32, shape=[], name='learning_rate')

xr=x
yr=y

base_scaler=64

#Going down
A1=leaky_relu(tf.layers.conv2d(xr, base_scaler, [3,1], padding='SAME', dilation_rate=1))
A2=leaky_relu(tf.layers.conv2d(A1, base_scaler, [3,1], padding='SAME', dilation_rate=2))
A3=leaky_relu(tf.layers.conv2d(A2, base_scaler, [3,1], padding='SAME', dilation_rate=3))
A4=leaky_relu(tf.layers.conv2d(A3, base_scaler, [3,1], padding='SAME', dilation_rate=4))
A5=leaky_relu(tf.layers.conv2d(A4, base_scaler, [3,1], padding='SAME', dilation_rate=5))
A6=leaky_relu(tf.layers.conv2d(A5, base_scaler, [3,1], padding='SAME', dilation_rate=6))
Flat=tf.layers.flatten(A4)

print(Flat.get_shape())

dense1=tf.layers.dense(Flat, units=5)
dense2=tf.layers.dense(dense1, units=2)
logits=leaky_relu(tf.layers.dense(dense2, units=1))

diff=tf.subtract(yr,logits)
LSQ=tf.multiply(diff,diff)
loss=tf.reduce_mean(LSQ)
train_op=tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

(?, 32000)


In [351]:
tf.set_random_seed(123456)
sess=tf.Session()
sess.run(tf.global_variables_initializer())
for f in range(0,5000):
    idx=np.random.choice(no_of_samples, replace=False, size=[400])
    cur_train=data[idx,:,:]
    cur_truth=truth[idx,:]
    _,results, losses=sess.run([train_op, logits, loss], feed_dict={x:cur_train, y:cur_truth, lr:0.001})
    if (f%100==0):
        res, valid_loss=sess.run([logits,loss], feed_dict={x:data_test, y:truth_test})
        print([f,np.sum(res), losses, valid_loss])
    

[0, 2793.8374, 0.38296103, 5.488803]
[100, 463.86743, 0.24091879, 0.24980493]
[200, 509.88956, 0.23452158, 0.24671988]
[300, 663.5762, 0.21745227, 0.27108657]
[400, 565.17224, 0.209082, 0.24803105]
[500, 618.8258, 0.18465322, 0.25385347]
[600, 498.90546, 0.16294636, 0.22108081]
[700, 483.75845, 0.12918392, 0.16190134]
[800, 508.74164, 0.058573533, 0.08381631]
[900, 453.13712, 0.031818673, 0.043534707]
[1000, 478.61844, 0.012360641, 0.02801716]
[1100, 425.31708, 0.011867799, 0.028781736]
[1200, 482.28906, 0.008696612, 0.01767406]
[1300, 419.86722, 0.023456817, 0.024732541]
[1400, 484.28296, 0.008987884, 0.013722212]
[1500, 476.96603, 0.006843269, 0.013547049]
[1600, 577.33826, 0.0043352596, 0.01791769]
[1700, 482.86713, 0.005374559, 0.011755422]
[1800, 430.19363, 0.005894857, 0.017720332]
[1900, 415.88452, 0.01221275, 0.020581055]
[2000, 534.53094, 0.004613353, 0.011345279]
[2100, 497.4315, 0.0030265294, 0.01054783]
[2200, 466.45016, 0.0035923219, 0.011667176]
[2300, 496.08557, 0.003241

In [352]:
results, losses=sess.run([logits, loss], feed_dict={x:data_test, y:truth_test})
np.where(results[:,0]>0.5)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [353]:
np.where(truth_test[:,0]==1)

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177